In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.chdir('..')

In [ ]:
# os.environ.get("ZHIPUAI_API_KEY")

# 设计和使用

## 命令清单

命令清单：
```python
_INVLIAD_COMMANDS = [
    "quit",       # 退出
    "text",       # 某ID下的文字成果，默认ROOT
    "all",        # 所有任务
    "todos",      # 所有待办
    "todo",       # 某ID待办，默认当前ID
    "ok",         # 确认INIT任务，或某ID提纲或段落，然后自动进入下一待办任务
    "children",   # 查看某ID提纲
    "words",      # 查看后修改某ID字数
    "title",      # 查看后修改某ID标题
    "howto",      # 查看后修改某ID扩写指南
    "summarise",  # 查看后修改某ID段落摘要
    "reload",     # 重新加载模型
    "memory",     # 某ID对话记忆，默认当前ID
    "store",      # 某ID对话历史，默认当前ID
    "ask",        # 向AI提问
]
```

## 概念

**关键概念**
- [x] focus概念：`START`, `<ID>`, `END`
- [x] 任务查看：所有，待办列表
- [x] 提示语变量中包含历史对话，默认支持20轮次
- [x] 每个节点有独立的对话历史
- [x] 内容ID：在同一个对象中全局自增
- [x] auto_mode: `all`全自动生成, `askme`每一步询问
- [x] run 的 task 参数
- [x] run 的 max_step参数
- [ ] 创作工程的状态机

**模型**
- [x] 支持自动识别 ZHIPUAI_API_KEY 和 OPENAI_API_KEY，并自动加载包和构造LLM对象
- [x] 也支持手动创建LLM对象

**高级模板管理**
- [ ] 提示语中的提纲信息：
  - 【模式1】 全路径选标题、扩写指南和内容摘要？
  - 【模式2】 祖先节点选标题，兄弟节点选标题、扩写指南和内容摘要，最近节点选文字详情？
- [ ] 提示语模板：任务提示、模板构成、扩写指南
- [ ] 提纲模板：从生成内容导出
- [ ] 提纲解析：从文件内容的层次结构中提取提纲结构，生成格式化提纲（包含标题、扩写指南等要素）

**导出**
- [ ] 文字合并：如何对待提纲中的标题，作为段落合并还是作为章节？

**关于修改：**
- [ ] 重新生成提纲列表：若要重新生成提纲列表，必须手工清楚所有子项，否则禁止使用todo指令或对其修改
- [ ] todo指令使用限制：仅针对子项尚未完成的对象（如刚刚完成的，或其他兄弟节点）
- [ ] 修改提纲子项属性：需要单独使用title、words、howto等指令修改


## Bug

Bug清单：
- [x] START 记忆确实存在，但无法通过 START@memory 正确提取
- [x] ask 指令无法在当前节点重新生成
- [x] todos清单增加一个标记为当前任务的状态：`[当前任务]`, `* <未完成>`, `<已完成>`
- [x] is_completed 无法修改


# WritingProject

## 每一步都询问：使用默认的智谱AI

In [2]:
from langchain_chinese import WritingTask

# 使用默认的智谱AI推理
wp = WritingTask(task_mode="askme")

In [3]:
wp.run("致孩子的一封信, 800字", max_steps=1)

[START]
Retry for HTTP Error ...
```json
{
    "总字数要求": 800,
    "标题名称": "致孩子的一封信：成长的寄语",
    "扩写指南": "本文是一封父母给孩子写的信，信中要表达对孩子的关爱、期望以及对其成长的鼓励和指导。信中可以提及孩子的兴趣、梦想以及父母的一些人生经验分享。建议在信中包含以下内容：对孩子的称呼；表达对孩子的思念和关心；对孩子的成长变化表示欣喜；分享一些重要的人生价值观；鼓励孩子面对困难勇敢向前；表达对孩子未来的美好祝愿。"
}
```


In [4]:
wp.run(max_steps=100)

[START]



👤:  ok


-------------------- TODOs --------------------
* [0] 约800字 | 《致孩子的一封信：成长的寄语》
```json
{
    "大纲列表": [
        {
            "总字数要求": 300,
            "标题名称": "温暖的关怀与思念",
            "扩写指南": "以亲切的称呼开头，描述对孩子的思念和关心，提及日常生活中的点滴细节，表达对孩子的深情。"
        },
        {
            "总字数要求": 200,
            "标题名称": "成长的喜悦与肯定",
            "扩写指南": "谈论孩子成长中的变化，强调对其进步的欣喜和对其个性的认可，鼓励孩子保持自我。"
        },
        {
            "总字数要求": 200,
            "标题名称": "人生经验与价值观",
            "扩写指南": "分享父母的人生经验，强调诚实、勇敢、善良等价值观的重要性，并指导孩子如何在日常生活中实践。"
        },
        {
            "总字数要求": 100,
            "标题名称": "勇敢面对困难",
            "扩写指南": "鼓励孩子面对挑战时保持积极态度，告诉他们困难是成长的必经之路，父母永远是他们坚强的后盾。"
        },
        {
            "总字数要求": 100,
            "标题名称": "未来的祝愿与期待",
            "扩写指南": "表达对孩子未来的美好祝愿，展望其成就和幸福，并以父母永恒的爱和支持作为结尾。"
        }
    ]
}
```



👤:  分为3部份即可


```json
{
    "大纲列表": [
        {
            "总字数要求": 350,
            "标题名称": "深情的关怀与思念",
            "扩写指南": "以亲昵的称呼开始，表达对孩子的思念和关心，描述日常生活中的温馨时刻，传递家庭的温暖和爱。"
        },
        {
            "总字数要求": 300,
            "标题名称": "成长的见证与寄语",
            "扩写指南": "分享孩子成长的喜悦和变化，肯定其独特性和成就，给出关于诚实、勇敢等价值观的指导，鼓励孩子追求梦想。"
        },
        {
            "总字数要求": 150,
            "标题名称": "未来的期望与祝福",
            "扩写指南": "表达对孩子未来的美好祝愿，鼓励其面对困难勇敢向前，并承诺父母永远的支持和爱。"
        }
    ]
}
```


...


👤:  0@todo


<0> is_completed False
[0]



👤:  all


-------------------- All --------------------
* [0] 800字以内 | 《致孩子的一封信：成长的寄语》
* <4> 350字以内 | 《深情的关怀与思念》
* <5> 300字以内 | 《成长的见证与寄语》
* <6> 150字以内 | 《未来的期望与祝福》



👤:  quit


In [ ]:
wp.run("memory")

In [ ]:
wp.run()

In [ ]:
wp.run("ok", max_steps=1)

In [ ]:
wp.run()

In [ ]:
wp.get_memory("START")

In [ ]:
wp.get_memory(0)

In [ ]:
wp.print_all()

## 全自动生成：使用默认的智谱AI

In [ ]:
from langchain_chinese import WritingTask

# 使用默认的智谱AI推理
# 你需要准备好智谱AI的APIKEY
wp = WritingTask(auto_mode="all")
wp.run("我今天扒了二牛的裤子，然后他一整天都不理我，帮我写一封1000字道歉信，我希望和他继续做好朋友，每段不要少于500字哦")

In [ ]:
x = """
1 事件经过
 那是一个玩笑过火的下午，我们一群朋友在公园里嬉闹，我因一时的顽皮与冲动，竟然在众目睽睽之下扒下了二牛的裤子。当时，我只是想着逗大家笑，却完全忽略了二牛的面子和感受。笑声中，我看到了二牛的尴尬和愤怒，那一刻，我突然意识到自己的行为是多么的愚蠢和错误。
2 自我反省
 在深入分析自己的行为背后的原因时，我意识到那天的举动源于一种愚蠢的冲动和对自己幽默感的过度自信。我一直以为，作为朋友，我们可以毫无顾忌地开玩笑。然而，我忽视了你的感受，更忽略了这样一个事实：真正的幽默绝不会以牺牲他人的尊严为代价。我现在深刻认识到，那天的行为不仅令你尴尬，更伤害了你的自尊。对此，我感到无比懊悔。
3 对二牛的歉意
 二牛，我想借此机会，用我最真诚的心意向你道歉。我深刻意识到在那次公园的尴尬事件中，我扒下你的裤子，给你带来了多大的伤害和不适。这不仅是一个简单的恶作剧，更是对你个人尊严的侵犯。我承认，这一行为严重影响了我们之间的友谊，我对此感到万分抱歉。我完全理解你在那一刻的感受，以及这件事可能给你带来的长期影响。我后悔莫及，如果时光可以倒流，我绝不会让那样的场面发生。
4 友谊的重要性
 二牛，自从公园那件事发生后，我一直在反思，你的友谊对我意味着什么。它不仅是快乐时光的分享，也是我在逆境中的支撑。我意识到，那天我的举动不仅仅是一个玩笑，它深深地伤害了你，也动摇了我们之间的信任基石。你对我来说非常重要，我珍视我们的友谊，它让我的生活更加丰富多彩。因此，我愿意不遗余力地去修复这段关系，证明我对这段友谊的重视和诚意。
5 改善关系的具体行动
 为了表达我的诚意和决心，我计划采取以下具体行动来改善我们的关系：首先，我会在第一时间私下向你道歉，面对面的交流可以让我更直接地传达我的悔意。其次，我会在我们共同的朋友圈中公开澄清这一事件的真相，说明这是我个人不当的行为，并非有意为之，以消除可能给你带来的负面影响。此外，我将改正自己的行为，承诺今后在公共场合保持适当的举止，不再让类似的事情发生。我还会主动参与一些能体现我对这段友谊重视的活动，比如一起参加你感兴趣的活动，或是在你需要帮助时伸出援手。通过这些具体的行动，我希望能够证明我对于这段友谊的珍视，以及对于过去错误行为的深刻反省。"""

len(x)

In [ ]:
wp.print_text()

In [ ]:
wp.get_memory("2")

## 查看记忆和对话历史

In [ ]:
[k for k in wp.memory._shorterm_memory_store]

In [ ]:
for x in wp.memory.get_shorterm_memory(session_id="20240508.180514.00002.626").chat_memory.messages:
    print("-"*40)
    print(x.content)

In [ ]:
wp.print_text()

## 全自动生成：更换为GPT4

In [ ]:
from langchain_openai import ChatOpenAI

# 使用 OpenAI/ChatGPT 推理
wp = WritingTask(task_mode="auto")
wp.run(llm = ChatOpenAI(model_name="gpt-4-turbo"))

In [ ]:
wp.print_lines()

In [ ]:
print(wp.root_content.get_outlines())

# 测试 WritingTask
## ask_user

In [ ]:
from langchain_chinese import WritingTask

# 使用默认的智谱AI推理
wp = WritingTask(task_mode="askme")
wp.ask_user("ok")

In [ ]:
wp.ask_user("1@ask 帮我按两个段落")

In [ ]:
wp.ask_user("START@ask 帮我按两个段落")

In [ ]:
wp.ask_user("ask 帮我按两个段落")

# 测试 TreeContent

In [ ]:
from langchain_chinese import TreeContent

root = TreeContent()
a = root.add_item(title="ABC")
b = root.add_item(title="XYZ")
print(a.id, a.title)
print(b.id, b.title)
print(root.id)
print(a.add_item().id)
print(b.add_item().id)
print(b.add_item().root.id)


In [ ]:
root.get_item_by_id("4").id

In [ ]:
root.get_item_by_id("3").id